In [28]:
# | default_exp scraper.Scraper

In [29]:
# | exporti
import re

import os
from dataclasses import dataclass
from typing import List, Callable, Any
from bs4 import BeautifulSoup


from urllib.parse import urljoin, urlparse

from concurrent.futures import ThreadPoolExecutor


from urllib.parse import urlparse
from gdoc_sync.utils import upsert_folder, convert_str_file_name

import gdoc_sync.scraper.driver as dg

In [30]:
# |exporti
from nbdev.showdoc import patch_to

In [31]:
# | hide
from selenium.webdriver.common.by import By
from pprint import pprint

In [32]:
# from selenium.webdriver.common.by import By

test_urls = [
    "https://domo-support.domo.com/s/article/36004740075?language=en_US",
    "https://domo-support.domo.com/s/topic/0TO5w000000ZlOmGAK/20202023?language=en_US",  # list of articles
    "https://domo-support.domo.com/s/topic/0TO5w000000Zan7GAC/archived-feature-release-notes?language=en_US",  # list of topics
    "https://domo-support.domo.com/s/knowledge-base?language=en_US",
]

drivergenerator = dg.DriverGenerator(debug_prn=True)

driver = drivergenerator.get_webdriver()

test_soup = dg.get_pagesource(
    driver=driver,
    url=test_urls[3],
    search_criteria_tuple=(By.CLASS_NAME, "topic-nav-container"),
    max_sleep_time=15,
    return_soup=True,
)

test_soup = dg.get_pagesource(
    driver=driver,
    url=test_urls[2],
    search_criteria_tuple=(
        By.CSS_SELECTOR,
        f".{', .'.join(['section-list-item', 'article-list-item'] )}",
    ),
    max_sleep_time=15,
    return_soup=True,
)

str(test_soup)[0:100]

ChromeDriver 120.0.6099.109 (3419140ab665596f21b385ce136419fde0924272-refs/branch-heads/6099@{#1483})



'<html dir="ltr" lang="en-US"><head><title>Archived Feature Release Notes</title><meta content="defau'

# Scrape_Config

In [33]:
# | export
def extract_links(
    soup: BeautifulSoup,
    base_url: str = None,
    custom_link_extractor_fn: Callable = None,
    debug_prn: bool = False,
) -> [str]:
    """returns a list of urls"""

    links_ls = []

    for link in soup.findAll("a"):
        if debug_prn:
            print(link)

        if link.has_attr("href"):
            url = link["href"]

            if custom_link_extractor_fn:
                url = custom_link_extractor_fn(link, base_url)

            if url:
                links_ls.append(url)

    return list(set(links_ls))

In [34]:
extract_links(
    test_soup,
    debug_prn=False,
    #   custom_link_extractor_fn= domokb_link_extractor_fn,
    #   base_url = "https://domo-support.domo.com"
)

['https://www.domo.com',
 'https://www.domo.com/start/developer',
 'https://innovators.domo.com',
 'https://www.domo.com/domo-central/community-awards',
 'https://developer.domo.com/login',
 'https://www.domo.com/domo-central/community',
 'https://learndomo.domo.com/pages/15/learn-by-skill-or-role',
 'archived-feature-release-notes?nocache=https%3A%2F%2Fdomo-support.domo.com%2Fs%2Ftopic%2F0TO5w000000Zan7GAC%2Farchived-feature-release-notes%3Flanguage%3Den_US',
 'https://www.domo.com/blog/category/domo-community',
 'https://www.domo.com/domo-central/help/videos',
 '/s/topic/0TO5w000000ZamwGAC',
 'https://www.domo.com/help-center/get-started',
 'https://linkedin.com/company/domo-inc-',
 'https://www.domo.com/domo-central/help/data-creators-and-analysts',
 '/s/topic/0TO5w000000ZlOmGAK',
 'https://community-forums.domo.com/main/categories',
 'https://community-forums.domo.com/main/categories/welcome',
 'https://university.domo.com/training/domo-trains-you',
 'https://facebook.com/domoHQ/ti

In [35]:
# | export
def generate_filename_from_url(url, base_folder=None, file_name=None):
    parsed_url = urlparse(url)

    base_str = "_".join([str for str in parsed_url[2].split("/") if str])

    if base_folder:
        base_str = os.path.join(base_folder, base_str)

    if file_name:
        base_str = os.path.join(base_str, file_name)

    return base_str

In [36]:
pprint([generate_filename_from_url(url) for url in test_urls])

print("-- alternative with base_url and different file name")
pprint([generate_filename_from_url(url, "/SCRAPE", "index.html") for url in test_urls])

['s_article_36004740075',
 's_topic_0TO5w000000ZlOmGAK_20202023',
 's_topic_0TO5w000000Zan7GAC_archived-feature-release-notes',
 's_knowledge-base']
-- alternative with base_url and different file name
['/SCRAPE/s_article_36004740075/index.html',
 '/SCRAPE/s_topic_0TO5w000000ZlOmGAK_20202023/index.html',
 '/SCRAPE/s_topic_0TO5w000000Zan7GAC_archived-feature-release-notes/index.html',
 '/SCRAPE/s_knowledge-base/index.html']


In [37]:
# |export
class ScrapeConfig_NoDriverProvided(Exception):
    def __init__(self, function_name=None):
        function_str = " to {function_name} " if function_name else ""
        super().__init__("Driver not provided" + function_str)


@dataclass
class Scrape_Config:
    """class for collating data about how to scrape a page"""

    pattern: re.Pattern  # url pattern
    link_extractor_fn: Callable = extract_links

    generate_filename_fn: Callable = generate_filename_from_url
    content_extractor_fn: Callable = None

    search_element_type: Any = None
    search_element_text: str = None

    max_sleep_time: int = 10

    url = None  # will be assigned after retrieved from scrape_factory
    download_folder = "./SCRAPE"

    scrape_crawler = None  # will be optional parent threadpool manager.

    def __id__(self, other):
        return self.pattern == other.pattern

    def get_search_tuple(self) -> Any:
        if not self.search_element_text and self.search_element_type:
            return None

        return (self.search_element_type, self.search_element_text)

    def is_text_match_pattern(self, text, debug_prn: bool = False):
        pattern = re.compile(self.pattern)

        match_pattern = pattern.match(text)

        if debug_prn:
            print({"text": text, "pattern": self.pattern})

        if not match_pattern:
            return False

        if debug_prn:
            print(match_pattern)

        return True

In [38]:
# | exporti


@patch_to(Scrape_Config)
def _get_pagesource(self: Scrape_Config, url: str, driver, debug_prn: bool = False):
    """handles fetching the pagesource, html content of a URL
    the classes' scrape_factory.get_factory_config() method will identify what to look for for that specific URL
    """

    search_criteria_tuple = self.get_search_tuple()
    max_sleep_time = self.max_sleep_time

    if debug_prn:
        print(
            {
                "url": url,
                "max_sleep_time": max_sleep_time,
                "search_criteria": search_criteria_tuple,
            }
        )

    pagesource = dg.get_pagesource(
        url=url,
        search_criteria_tuple=search_criteria_tuple,
        driver=driver,
        max_sleep_time=max_sleep_time,
    )

    if not pagesource:
        raise Exception(f"unable to retrieve source {url}")

    return pagesource


@patch_to(Scrape_Config)
def _download_content(self: Scrape_Config, file_name, content):
    dir_name = os.path.dirname(file_name)

    if dir_name[-1] != "/":
        dir_name += "/"

    upsert_folder(dir_name)

    with open(file_name, "w", encoding="utf-8") as f:
        f.write(str(content))

    return file_name

# Scrape Factory converts URL into Scrape_Config task

In [54]:
# | export


class Scrape_Factory_NoConfigMatch(Exception):
    def __init__(self, text):
        super().__init__(
            f"{text} has no pattern match in factory_configs, add an appropriate config or check pattern matches"
        )


class ScrapeFactory_NoBaseUrl(Exception):
    def __init__(self):
        super().__init__(
            "no base_url provided to scrape_factory. unable to extract relevant URLs"
        )


@dataclass
class Scrape_Factory:
    """class handles a list of Scrape_Configs and will return the 'correct one' given a URL"""

    factory_configs: List[Scrape_Config]

    def get_factory_config(
        self,
        url,
        base_url=None,
        download_folder="./SCRAPE",
        scrape_crawler=None,
        debug_prn: bool = False,
    ):
        config = next(
            (
                config
                for config in self.factory_configs
                if config.is_text_match_pattern(url, debug_prn=debug_prn)
            ),
            None,
        )

        if not config:
            raise Scrape_Factory_NoConfigMatch(text=url)

        config.url = url
        config.download_folder = download_folder
        config.base_url = (
            base_url or (scrape_crawler and scrape_crawler.base_url) or None
        )
        config.scrape_crawler = scrape_crawler

        return config

In [40]:
# | exporti


@patch_to(Scrape_Config)
def scrape_page(
    self: Scrape_Config,
    url=None,  # can be passed in or retrieved from class after it has been converted into a task (been returned by scrape_factory)
    base_url=None,
    driver=None,
    debug_prn: bool = False,
    is_suppress_errors: bool = False,
    is_test: bool = False,
):
    url = url or self.url

    driver = (
        driver
        or (
            self.scrape_crawler
            and self.scape_crawler.driver_generator
            and self.scrape_crawler.driver_generator.get_webdriver()
        )
        or None
    )

    if not driver:
        raise ScrapeConfig_NoDriverProvided()

    if debug_prn:
        print(f"scraping_page {url}")

    try:
        soup = self._get_pagesource(url=url, driver=driver, debug_prn=debug_prn)

        file_name = self.generate_filename_fn(
            url=self.url, base_folder=self.download_folder, file_name="index.html"
        )

        save_location = self._download_content(file_name=file_name, content=soup)

        # Extract the article content
        if self.content_extractor_fn:
            content = self.content_extractor_fn(soup)

            content_name = self.generate_filename_fn(
                url=self.url, base_folder=self.download_folder, file_name="content.html"
            )

            self._download_content(
                content_name,
                content=content,
            )

        print(f"🎉 successfully scraped {url} to {save_location}")

        if is_test:
            return True

        if not is_test:
            base_url = (
                base_url
                or self.base_url
                or (self.scrape_crawler and self.scape_crawler.base_url)
                or None
            )

            if not base_url:
                raise ScrapeFactory_NoBaseUrl()
            """if config is part of a scrape_crawler / managed threadpool will update crawler's urls_to_visit list."""

            return self.link_extractor_fn(soup, base_url)

    except Exception as e:
        if not is_suppress_errors:
            raise (e)
        return f"💀 failed to download {url} received errror{e}"

# DomoKB_ScrapeConfig

In [41]:
# | exporti
def process_domo_kb_link(link, base_url):
    url = link.get("href")

    if not url:
        return None

    # for relative addresses concat base_url
    if url.startswith("/s/"):
        url = urljoin(base_url, url)

    # ignore urls not orginating from base_url
    if not url.startswith(base_url):
        return None

    # remove query params
    url = urljoin(url, urlparse(url).path)

    # only keep the first 6 pieces of the URL
    url = "/".join(url.split("/")[:6])

    if url.endswith("/"):
        url = url[:-1]

    return url


def domokb_link_extractor_fn(soup, base_url, debug_prn: bool = False):
    return extract_links(
        soup,
        custom_link_extractor_fn=process_domo_kb_link,
        base_url=base_url,
        debug_prn=debug_prn,
    )

In [42]:
domokb_link_extractor_fn(test_soup, "https://domo-support.domo.com")

['https://domo-support.domo.com/s/topic/0TO5w000000ZlOmGAK',
 'https://domo-support.domo.com/s/knowledge-base',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZlOnGAK',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZamwGAC']

## DomoKB_ScrapeConfig_Article

In [43]:
# | exporti


def domokb_article_content_extractor_fn(soup) -> BeautifulSoup:
    return soup.find(class_=["article-column"])


DomoKB_ScrapeConfig_Article = Scrape_Config(
    pattern=r".*/s/article/.*",
    link_extractor_fn=domokb_link_extractor_fn,
    content_extractor_fn=domokb_article_content_extractor_fn,
    search_element_type=By.CLASS_NAME,
    search_element_text="slds-form-element",
)

## DomoKB_ScrapeConfig_Topic

In [44]:
# | exporti
def domokb_topic_content_extractor_fn(soup) -> BeautifulSoup:
    return soup.find(class_=["knowledge-base"])


DomoKB_ScrapeConfig_Topic = Scrape_Config(
    pattern=r".*/s/topic/.*",
    link_extractor_fn=domokb_link_extractor_fn,
    content_extractor_fn=domokb_topic_content_extractor_fn,
    search_element_type=By.CSS_SELECTOR,
    search_element_text=f".{', .'.join(['section-list-item', 'article-list-item'] )}",
)

## DomoKB_ScrapeConfig_NavContainer
cDomoKBCategoryNav"

In [45]:
# |exporti
def domokb_knowledgebase_content_extractor_fn(soup) -> BeautifulSoup:
    return soup.find(class_=["knowledge-base"])


DomoKB_ScrapeConfig_KnowledgeBase = Scrape_Config(
    pattern=r".*/s/knowledge-base.*",
    link_extractor_fn=domokb_link_extractor_fn,
    content_extractor_fn=domokb_knowledgebase_content_extractor_fn,
    search_element_type=By.CSS_SELECTOR,
    search_element_text=f".{', .'.join(['topic-nav-container', 'cDomoKBCategoryNav'] )}",
)

## DomoKB_ScrapeFactory
A factory pattern receives a parameter (the url to match to) then returns the appropriate implementation `Scrape_Config` for handling that url.

This pattern could be extended for any type of website; however we have implemented a factory specifically for handling Domo Kbs

In [46]:
# | export
DomoKB_ScrapeFactory = Scrape_Factory(
    [
        DomoKB_ScrapeConfig_Article,
        DomoKB_ScrapeConfig_Topic,
        DomoKB_ScrapeConfig_KnowledgeBase,
    ]
)

#### sample implementation

Of retrieving the scrape_config using the `scrape_factory.get_factory_config() method

- test_urls is a list of URLS
- use a list comprenension to show we can retrieve the correct config for different URLs

In [47]:
test_scrape_tasks = [
    {
        "url": url,
        "config": DomoKB_ScrapeFactory.get_factory_config(
            url, base_url="https://domo-support.domo.com", debug_prn=False
        ),
    }
    for url in test_urls
]

test_scrape_tasks

[{'url': 'https://domo-support.domo.com/s/article/36004740075?language=en_US',
  'config': Scrape_Config(pattern='.*/s/article/.*', link_extractor_fn=<function domokb_link_extractor_fn at 0x7f793342b6a0>, generate_filename_fn=<function generate_filename_from_url at 0x7f7932e714e0>, content_extractor_fn=<function domokb_article_content_extractor_fn at 0x7f7932e71da0>, search_element_type='class name', search_element_text='slds-form-element', max_sleep_time=10)},
 {'url': 'https://domo-support.domo.com/s/topic/0TO5w000000ZlOmGAK/20202023?language=en_US',
  'config': Scrape_Config(pattern='.*/s/topic/.*', link_extractor_fn=<function domokb_link_extractor_fn at 0x7f793342b6a0>, generate_filename_fn=<function generate_filename_from_url at 0x7f7932e714e0>, content_extractor_fn=<function domokb_topic_content_extractor_fn at 0x7f79b67dc5e0>, search_element_type='css selector', search_element_text='.section-list-item, .article-list-item', max_sleep_time=10)},
 {'url': 'https://domo-support.domo

In [48]:
test_task = test_scrape_tasks[2]["config"]
test_task.__dict__

{'pattern': '.*/s/topic/.*',
 'link_extractor_fn': <function __main__.domokb_link_extractor_fn(soup, base_url, debug_prn: bool = False)>,
 'generate_filename_fn': <function __main__.generate_filename_from_url(url, base_folder=None, file_name=None)>,
 'content_extractor_fn': <function __main__.domokb_topic_content_extractor_fn(soup) -> bs4.BeautifulSoup>,
 'search_element_type': 'css selector',
 'search_element_text': '.section-list-item, .article-list-item',
 'max_sleep_time': 10,
 'url': 'https://domo-support.domo.com/s/topic/0TO5w000000Zan7GAC/archived-feature-release-notes?language=en_US',
 'download_folder': './SCRAPE',
 'base_url': 'https://domo-support.domo.com',
 'scrape_crawler': None}

In [49]:
driver_generator = dg.DriverGenerator()
driver = driver_generator.get_webdriver()

test_task.scrape_page(driver=driver)

🎉 successfully scraped https://domo-support.domo.com/s/topic/0TO5w000000Zan7GAC/archived-feature-release-notes?language=en_US to ./SCRAPE/s_topic_0TO5w000000Zan7GAC_archived-feature-release-notes/index.html


['https://domo-support.domo.com/s/topic/0TO5w000000ZlOmGAK',
 'https://domo-support.domo.com/s/knowledge-base',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZlOnGAK',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZamwGAC']

# Scrape_Crawler

The Scrape_Crawler is generally just a crawler that receieves a URL then manages scraping that page (by retrieving the correct scrape_configuration process) and searching for URLs to extract from that page and adding it to the `urls_to_visit` property.

The ThreadPoolExecutor has a maximum number of threads, maximum_workers, it will use.

This crawler does not concern itself with the return of the webcrawl task, because the task handles downloading the HTML file to a parameterized location 

In [50]:
# | export


@dataclass
class Scrape_Crawler:
    """threadpool manager for crawling through a list of urls"""

    executor: ThreadPoolExecutor
    scrape_factory: Scrape_Factory
    base_url: str

    download_folder: str
    visited_urls: set
    urls_to_visit: set

    driver_generator: dg.DriverGenerator = None

    def __init__(
        self,
        driver_path,
        scrape_factory: Scrape_Factory,
        urls_to_visit: list,
        base_url: str,
        urls_visited: list = None,
        max_workers=5,
        download_folder: str = "./SCRAPE/",
    ):
        self.base_url = base_url
        self.scrape_factory = scrape_factory
        self.executor = ThreadPoolExecutor(max_workers=max_workers)
        self.driver_generator = dg.DriverGenerator(driver_path=driver_path)
        self.download_folder = download_folder

        self.visited_urls = set()
        if urls_visited:
            [self._add_url_to_visited(url) for url in urls_visited]

        self.urls_to_visit = set()

        if urls_to_visit:
            [self._add_url_to_visit(url) for url in urls_to_visit]

    def _add_url_to_visit(self, url, debug_prn: bool = False):
        """adds a URL to the list of URLS to visit after testing that the URL has not already been visited"""

        if url not in self.visited_urls and url not in self.urls_to_visit:
            if debug_prn:
                print(f"adding {url} to to_vist list")

            self.urls_to_visit.add(url)
            return self.urls_to_visit

    def _add_url_to_visited(self, url, debug_prn: bool = False):
        if url not in self.visited_urls:
            if debug_prn:
                print(f"adding {url} to visited list")

            self.visited_urls.add(url)
            return self.visited_urls

    def _quit(self):
        """call when the executor queue is empty"""

        self.executor.shutdown(wait=True)
        return f"Done scraping {len(self.visited_urls)} urls"

In [51]:
wbs = Scrape_Crawler(
    driver_path="/usr//bin/chromedriver",
    scrape_factory=DomoKB_ScrapeFactory,
    base_url="https://domo-support.domo.com",
    urls_to_visit=test_urls,
)
wbs.__dict__

{'base_url': 'https://domo-support.domo.com',
 'scrape_factory': Scrape_Factory(factory_configs=[Scrape_Config(pattern='.*/s/article/.*', link_extractor_fn=<function domokb_link_extractor_fn at 0x7f793342b6a0>, generate_filename_fn=<function generate_filename_from_url at 0x7f7932e714e0>, content_extractor_fn=<function domokb_article_content_extractor_fn at 0x7f7932e71da0>, search_element_type='class name', search_element_text='slds-form-element', max_sleep_time=10), Scrape_Config(pattern='.*/s/topic/.*', link_extractor_fn=<function domokb_link_extractor_fn at 0x7f793342b6a0>, generate_filename_fn=<function generate_filename_from_url at 0x7f7932e714e0>, content_extractor_fn=<function domokb_topic_content_extractor_fn at 0x7f79b67dc5e0>, search_element_type='css selector', search_element_text='.section-list-item, .article-list-item', max_sleep_time=10), Scrape_Config(pattern='.*/s/knowledge-base.*', link_extractor_fn=<function domokb_link_extractor_fn at 0x7f793342b6a0>, generate_filenam

In [52]:
# | exporti


@patch_to(Scrape_Crawler)
def crawl_urls(self: Scrape_Crawler, is_test: bool = False, debug_prn: bool = False):
    while self.urls_to_visit:
        url = self.urls_to_visit.pop()
        self.visited_urls.add(url)

        driver = self.driver_generator.get_webdriver()

        config = self.scrape_factory.get_factory_config(
            url, debug_prn=debug_prn, scrape_crawler=self
        )

        future = self.executor.submit(
            config.scrape_page, driver=driver, is_test=is_test
        )
        try:
            urls_to_visit, visited_urls = future.result()

            print(urls_to_visit, visited_urls)

            if urls_to_visit:
                [self._add_url_to_visit(url) for url in urls_to_visit]

            if visited_urls:
                [self._add_url_to_visited(url) for url in visited_urls]

        except Exception as e:
            print(e)

    return self._quit()

In [53]:
wbs = Scrape_Crawler(
    driver_path="/usr//bin/chromedriver",
    scrape_factory=DomoKB_ScrapeFactory,
    base_url="https://domo-support.domo.com",
    urls_to_visit=test_urls,
)

wbs.crawl_urls()

wbs.__dict__

TypeError: Scrape_Factory.get_factory_config() missing 1 required positional argument: 'base_url'

In [ ]:
# | hide
# import nbdev

# nbdev.nbdev_export()